In [1]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1  FALSE POSITIVE              0              1              0              0   
2  FALSE POSITIVE              0              1              0              0   
3       CONFIRMED              0              0              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
0   54.418383     2.479000e-04    -2.479000e-04   162.513840   
1   19.899140     1.490000e-05    -1.490000e-05   175.850252   
2    1.736952     2.630000e-07    -2.630000e-07   170.307565   
3    2.525592     3.760000e-06    -3.760000e-06   171.595550   
4    4.134435     1.050000e-05    -1.050000e-05   172.979370   

   koi_time0bk_err1  ...  koi_steff_err2  koi_slogg  koi_slogg_err1  \
0          0.003520  ...             -81      4.467           0.064   
1          0.000581  ...            -176      4.544           0.044   
2          0.000115  ...            -174      4.564           0.053   
3          0.001130  ...            -211      4.438           0.070   
4          0.001900  ...            -232      4.486           0.054   

   koi_slogg_err2  koi_srad  koi_srad_err1  koi_srad_err2         ra  \
0          -0.096     0.927          0.105         -0.061  291.93423   
1          -0.176     0.868          0.233         -0.078  297.00482   
2          -0.168     0.791          0.201         -0.067  285.53461   
3          -0.210     1.046          0.334         -0.133  288.75488   
4          -0.229     0.972          0.315         -0.105  296.28613   

         dec  koi_kepmag  
0  48.141651      15.347  
1  48.134129      15.436  
2  48.285210      15.597  
3  48.226200      15.509  
4  48.224670      15.714  

[5 rows x 41 columns]

# Select your features (columns)

In [29]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt','koi_fpflag_ss','koi_fpflag_co','koi_fpflag_ec','koi_period','koi_time0bk',
                        'koi_impact','koi_duration','koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr',
                        'koi_tce_plnt_num','koi_steff','koi_slogg','koi_srad','ra','dec','koi_kepmag']]
feature_names = selected_features.columns
feature_names

Index(['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
       'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth',
       'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_tce_plnt_num',
       'koi_steff', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag'],
      dtype='object')

# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
# Set disposition for y
target = df["koi_disposition"]
target_names = ["CANDIDATE", "CONFIRMED", "FALSE POSITIVE"]

#data = target.copy()
#data_binary_encoded = pd.get_dummies(data)
#data_binary_encoded.head()

CANDIDATE  CONFIRMED  FALSE POSITIVE
0          0          1               0
1          0          0               1
2          0          0               1
3          0          1               0
4          0          1               0

In [15]:
# Test split data, test data = 20%
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(selected_features, data_binary_encoded, test_size=0.20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(selected_features, target, test_size=0.20, random_state=42)

In [16]:
X_train.head()

koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  koi_period  \
3875              1              0              0              0   11.377528   
1768              0              0              0              0    2.215713   
3250              0              0              0              0    7.785911   
6574              1              0              1              1    2.404557   
2815              0              0              0              0  110.461746   

      koi_time0bk  koi_impact  koi_duration  koi_depth  koi_prad  koi_teq  \
3875    132.58480       1.415       12.9180     1651.3     82.26     1177   
1768    131.82107       0.796        1.8211      285.1      1.70     1283   
3250    137.87340       0.675        7.7880       45.5      0.68      939   
6574    131.67616       0.970        7.4918      408.7      4.69     1958   
2815    147.54650       0.036       12.4600      331.2      1.93      420   

      koi_insol  koi_model_snr  koi_tce_plnt_num  koi_steff  koi_slogg  \
3875     453.61           38.2                 1       6943      4.162   
1768     638.56           22.1                 1       5417      4.447   
3250     183.92           12.0                 1       6045      4.479   
6574    3474.41          147.1                 1       6426      4.013   
2815       7.34           15.8                 1       6321      4.434   

      koi_srad         ra        dec  koi_kepmag  
3875     1.660  300.19409  45.145741      13.930  
1768     0.901  297.02008  43.432549      15.392  
3250     0.954  290.42307  51.388729      13.515  
6574     1.775  293.21356  46.175129      13.474  
2815     1.058  287.03952  46.481701      15.092

In [17]:
y_train.head()

3875    FALSE POSITIVE
1768         CONFIRMED
3250         CANDIDATE
6574    FALSE POSITIVE
2815         CANDIDATE
Name: koi_disposition, dtype: object

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [22]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)

C:\Users\j6921\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [24]:
X_train_scaled

array([[1.        , 0.        , 0.        , ..., 0.9306672 , 0.54372496,
        0.57558476],
       [0.        , 0.        , 0.        , ..., 0.78542453, 0.43501043,
        0.69642119],
       [0.        , 0.        , 0.        , ..., 0.48354539, 0.93988811,
        0.5412844 ],
       ...,
       [0.        , 1.        , 0.        , ..., 0.792823  , 0.59720043,
        0.30217373],
       [1.        , 0.        , 0.        , ..., 0.77821733, 0.29000226,
        0.52153071],
       [0.        , 0.        , 1.        , ..., 0.68082222, 0.27185353,
        0.61930738]])

# Train the Model



In [34]:
#from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train)
#print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
#print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")
a = rf.score(X_train, y_train)
b = rf.score(X_test, y_test)
print(f"{a}, {b}")

0.7446351931330472, 0.7140814867762688


In [35]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.12992853634238974, 'koi_model_snr'),
 (0.12850808904830863, 'koi_fpflag_nt'),
 (0.12046443401510333, 'koi_fpflag_co'),
 (0.0921726378139902, 'koi_fpflag_ss'),
 (0.08510239930486177, 'koi_prad'),
 (0.048941227687465574, 'koi_depth'),
 (0.047704061855054686, 'koi_fpflag_ec'),
 (0.045813015032896226, 'koi_period'),
 (0.040244377466311466, 'koi_impact'),
 (0.038022624744200796, 'koi_teq'),
 (0.03063893930504415, 'koi_duration'),
 (0.030113426935610522, 'koi_time0bk'),
 (0.026643042008808866, 'koi_insol'),
 (0.023324229913071523, 'koi_steff'),
 (0.022367622385501957, 'ra'),
 (0.02156080580679381, 'dec'),
 (0.020421609520532152, 'koi_kepmag'),
 (0.02036791303706832, 'koi_srad'),
 (0.019857470787897295, 'koi_slogg'),
 (0.0078035369890888575, 'koi_tce_plnt_num')]

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)